<a href="https://colab.research.google.com/github/AsmaaAlghamd1/Asmaa_INFO5731_-Spring2021/blob/main/Alghamdi_In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:

import pandas as pd 
import re 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
from sklearn.model_selection import train_test_split
import numpy
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:

import pandas as pd
Train_data = open('/content/stsa-train.txt').read()
Train_label, Train_list = [], []
for i, j in enumerate(Train_data.split("\n")):
    c = j.split(' ')
    Train_label.append(c[0])
    Train_list.append(" ".join(c[1:]))



In [ ]:
Test_data = open('/content/stsa-test.txt').read()
Test_label, Test_list = [], []
for i, j in enumerate(test_data.split("\n")):
    c = j.split(' ')
    Test_label.append(c[0])
    Test_list.append(" ".join(c[1:]))

In [ ]:
# creating data frame
df = pd.DataFrame (list(zip(Train_list, Train_label)) , columns = ['Text', 'Labels'])
Data_df= pd.DataFrame (list(zip(Test_list, Test_label)) , columns = ['Text', 'Labels'])
Data_df = Data_df.dropna()
df


,Text,Labels
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0
6919,a deliciously nonsensical comedy about a city ...,1


In [ ]:
Data_df

,Text,Labels
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1
...,...,...
1817,the problem with concept films is that if the ...,0
1818,"safe conduct , however ambitious and well-inte...",0
1819,"a film made with as little wit , interest , an...",0
1820,"but here 's the real damn : it is n't funny , ...",0


In [ ]:

from textblob import Word
nltk.download('wordnet')
st = PorterStemmer()
nltk.download('punkt')
stop = stopwords.words('english')
#Cleaning train data
#Lower case
df['Lowercasing'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation
df['Punctuation Removals'] = df['Lowercasing'].str.replace('[^\w\s]','')
#Removing special charachters
df['Aspecial charachters Removals'] = df['Punctuation Removals'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
#Removing stopwords
df['Stopwords Removals'] = df['Punctuation Removals'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Tokenization
df['Tokenization'] = df['Stopwords Removals'].apply(lambda x: TextBlob(x).words)
#Lemmatization
df['Lemmatization'] = df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df

,Text,Labels,Lowercasing,Punctuation Removals,Punctuation Removals,Aspecial charachters Removals,Stopwords Removals,Tokenization,Lemmatization
0,"a stirring , funny and finally transporting re...",1,"a stirring , funny and finally transporting re...",a stirring funny and finally transporting rei...,a stirring funny and finally transporting rei...,a stirring funny and finally transporting rei...,stirring funny finally transporting reimaginin...,"[stirring, funny, finally, transporting, reima...",stirring funny finally transporting reimaginin...
1,apparently reassembled from the cutting-room f...,0,apparently reassembled from the cutting-room f...,apparently reassembled from the cuttingroom fl...,apparently reassembled from the cuttingroom fl...,apparently reassembled from the cuttingroom fl...,apparently reassembled cuttingroom floor given...,"[apparently, reassembled, cuttingroom, floor, ...",apparently reassembled cuttingroom floor given...
2,they presume their audience wo n't sit still f...,0,they presume their audience wo n't sit still f...,they presume their audience wo nt sit still fo...,they presume their audience wo nt sit still fo...,they presume their audience wo nt sit still fo...,presume audience wo nt sit still sociology les...,"[presume, audience, wo, nt, sit, still, sociol...",presume audience wo nt sit still sociology les...
3,this is a visually stunning rumination on love...,1,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,this is a visually stunning rumination on love...,visually stunning rumination love memory histo...,"[visually, stunning, rumination, love, memory,...",visually stunning rumination love memory histo...
4,jonathan parker 's bartleby should have been t...,1,jonathan parker 's bartleby should have been t...,jonathan parker s bartleby should have been th...,jonathan parker s bartleby should have been th...,jonathan parker s bartleby should have been th...,jonathan parker bartleby beallendall modernoff...,"[jonathan, parker, bartleby, beallendall, mode...",jonathan parker bartleby beallendall modernoff...
...,...,...,...,...,...,...,...,...,...
6916,take care is nicely performed by a quintet of ...,0,take care is nicely performed by a quintet of ...,take care is nicely performed by a quintet of ...,take care is nicely performed by a quintet of ...,take care is nicely performed by a quintet of ...,take care nicely performed quintet actresses n...,"[take, care, nicely, performed, quintet, actre...",take care nicely performed quintet actress non...
6917,"the script covers huge , heavy topics in a bla...",0,"the script covers huge , heavy topics in a bla...",the script covers huge heavy topics in a blan...,the script covers huge heavy topics in a blan...,the script covers huge heavy topics in a blan...,script covers huge heavy topics bland surfacey...,"[script, covers, huge, heavy, topics, bland, s...",script cover huge heavy topic bland surfacey w...
6918,a seriously bad film with seriously warped log...,0,a seriously bad film with seriously warped log...,a seriously bad film with seriously warped log...,a seriously bad film with seriously warped log...,a seriously bad film with seriously warped log...,seriously bad film seriously warped logic writ...,"[seriously, bad, film, seriously, warped, logi...",seriously bad film seriously warped logic writ...
6919,a deliciously nonsensical comedy about a city ...,1,a deliciously nonsensical comedy about a city ...,a deliciously nonsensical comedy about a city ...,a deliciously nonsensical comedy about a city ...,a deliciously nonsensical comedy about a city ...,deliciously nonsensical comedy city coming apa...,"[deliciously, nonsensical, comedy, city, comin...",deliciously nonsensical comedy city coming apa...


In [ ]:
#cleaning test data
Data_df['Lowercasing'] = Data_df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing punctuation
Data_df['Punctuation Removals'] = Data_df['Lowercasing'].str.replace('[^\w\s]','')
#Removing special characters
Data_df[' Special Characters Removals'] = Data_df['Punctuation Removals'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
#Removing stopwords
Data_df['Stopwords Removals'] = Data_df['Punctuation Removals'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Tokenization
Data_df['Tokenization'] = Data_df['Stopwords Removals'].apply(lambda x: TextBlob(x).words)
#Lemmatization
Data_df['Lemmatization'] = Data_df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

In [ ]:
Data_df

,Text,Labels,Lowercasing,Punctuation Removals,Special Characters Removals,Stopwords Removals,Tokenization,Lemmatization
0,"no movement , no yuks , not much of anything .",0,"no movement , no yuks , not much of anything .",no movement no yuks not much of anything,no movement no yuks not much of anything,movement yuks much anything,"[movement, yuks, much, anything]",movement yuks much anything
1,"a gob of drivel so sickly sweet , even the eag...",0,"a gob of drivel so sickly sweet , even the eag...",a gob of drivel so sickly sweet even the eage...,a gob of drivel so sickly sweet even the eage...,gob drivel sickly sweet even eager consumers m...,"[gob, drivel, sickly, sweet, even, eager, cons...",gob drivel sickly sweet even eager consumer mo...
2,"gangs of new york is an unapologetic mess , wh...",0,"gangs of new york is an unapologetic mess , wh...",gangs of new york is an unapologetic mess who...,gangs of new york is an unapologetic mess who...,gangs new york unapologetic mess whose saving ...,"[gangs, new, york, unapologetic, mess, whose, ...",gang new york unapologetic mess whose saving g...
3,"we never really feel involved with the story ,...",0,"we never really feel involved with the story ,...",we never really feel involved with the story ...,we never really feel involved with the story ...,never really feel involved story ideas remain ...,"[never, really, feel, involved, story, ideas, ...",never really feel involved story idea remain a...
4,this is one of polanski 's best films .,1,this is one of polanski 's best films .,this is one of polanski s best films,this is one of polanski s best films,one polanski best films,"[one, polanski, best, films]",one polanski best film
...,...,...,...,...,...,...,...,...
1817,the problem with concept films is that if the ...,0,the problem with concept films is that if the ...,the problem with concept films is that if the ...,the problem with concept films is that if the ...,problem concept films concept poor one saving ...,"[problem, concept, films, concept, poor, one, ...",problem concept film concept poor one saving m...
1818,"safe conduct , however ambitious and well-inte...",0,"safe conduct , however ambitious and well-inte...",safe conduct however ambitious and wellintent...,safe conduct however ambitious and wellintent...,safe conduct however ambitious wellintentioned...,"[safe, conduct, however, ambitious, wellintent...",safe conduct however ambitious wellintentioned...
1819,"a film made with as little wit , interest , an...",0,"a film made with as little wit , interest , an...",a film made with as little wit interest and ...,a film made with as little wit interest and ...,film made little wit interest professionalism ...,"[film, made, little, wit, interest, profession...",film made little wit interest professionalism ...
1820,"but here 's the real damn : it is n't funny , ...",0,"but here 's the real damn : it is n't funny , ...",but here s the real damn it is nt funny either,but here s the real damn it is nt funny either,real damn nt funny either,"[real, damn, nt, funny, either]",real damn nt funny either


In [ ]:
#Data Transformation
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
vector_train = TfidfVectorizer(analyzer='word')
vector_train.fit(df['Lemmatization'])
Tfidf =  vector_train.transform(df['Lemmatization'])





In [ ]:

vector_test = TfidfVectorizer(analyzer='word', vocabulary = vector_train.vocabulary_)
vector_test.fit(Data_df['Lemmatization'])
xtest = vector_test.transform(Data_df['Lemmatization'])
test_y = Data_df['Labels']

In [ ]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(Tfidf,df['Labels'].values,test_size=0.2)

In [ ]:

from sklearn.model_selection import cross_val_score, KFold
def validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  CV = cross_val_score(modelName, x, y, cv=kfold).mean()
  return CV

In [ ]:
from sklearn import metrics
def Metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1

## *MultinominalNB*

In [ ]:
NB_model = naive_bayes.MultinomialNB()
NB_model.fit(train_x, train_y)
NB_predictions= NB_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(NB_predictions, valid_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
print("_____________")
NB_scores = validation_score(naive_bayes_model, valid_x, valid_y)
print('Navie Bayes Cross Validation Score is {0}'.format(NB_scores))

Accuracy is 0.7653429602888087
Precision is 0.7653429602888087
Recall is 0.7653429602888087
F1 is 0.7653429602888087
_____________
Navie Bayes Cross Validation Score is 0.686695860702742


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [ ]:

NB_predict_test = NB_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(NB_predict_test, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))
print("_____________")
NBtest_score = validation_score(naive_bayes_model, xtest, test_y)
print('Navie Bayes Cross Validation Score is: {0}'.format(NBtest_score))

Accuracy is 0.7952799121844127
Precision is 0.7952799121844127
Recall is 0.7952799121844127
F1 is 0.7952799121844127
_____________
Navie Bayes Cross Validation Score is: 0.7371044256290158


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


# (SVM) Validation Metrics

In [ ]:
SVM_model = svm.SVC()
SVM_model.fit(train_x, train_y)
SVM_predictions = SVM_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(SVM_predictions, valid_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
SVM_score = validation_score(svm_model, valid_x, valid_y)
print('SVM Cross Validation Score is: {0}'.format(SVM_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.7732851985559567
Precision is: 0.7732851985559567
Recall is: 0.7732851985559567
F1 is: 0.7732851985559568
_____________
SVM Cross Validation Score is: 0.6693149827963716


In [ ]:
SVM_predict_test = SVM_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(SVM_predict_test, test_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
SVMtest_score = validation_score(SVM_model, xtest, test_y)
print('SVM Cross Validation Score is: {0}'.format(SVMtest_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.7936333699231614
Precision is: 0.7936333699231614
Recall is: 0.7936333699231614
F1 is: 0.7936333699231614
_____________
SVM Cross Validation Score is: 0.7239086050561461


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(n_neighbors = 15)
KNN_model.fit(train_x, train_y)
KNN_predictions = KNN_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(KNN_predictions, valid_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
KNN_score = validation_score(KNN_model, valid_x, valid_y)
print('KNN Cross Validation Score is: {0}'.format(KNN_score))

Accuracy is: 0.7227436823104693
Precision is: 0.7227436823104693
Recall is: 0.7227436823104693
F1 is: 0.7227436823104693
_____________
KNN Cross Validation Score is: 0.646105724116359


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [ ]:
KNN_predict_test = KNN_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(KNN_predict_test, test_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
KNNtest_score = validation_score(knn_model, xtest, test_y)
print('KNN Cross Validation Score is: {0}'.format(KNNtest_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.7354555433589463
Precision is: 0.7354555433589463
Recall is: 0.7354555433589463
F1 is: 0.7354555433589463
_____________
KNN Cross Validation Score is: 0.6361376328589443


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier()
DT_model.fit(train_x, train_y)
DT_predictions = DT_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(DT_predictions, valid_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
DT_score = validation_score(DT_model, valid_x, valid_y)
print('Decision Tree Cross Validation Score is: {0}'.format(DT_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.6404332129963899
Precision is: 0.6404332129963899
Recall is: 0.6404332129963899
F1 is: 0.6404332129963899
_____________
Decision Tree Cross Validation Score is: 0.5262954853508497


In [ ]:
DT_predict_test= DT_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(DT_predict_test, test_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
DTtest_score = validation_score(DT_model, xtest, test_y)
print('Decision Tree Cross Validation Score is: {0}'.format(DTtest_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.6734357848518112
Precision is: 0.6734357848518112
Recall is: 0.6734357848518112
F1 is: 0.6734357848518112
_____________
Decision Tree Cross Validation Score is: 0.6163183810724794


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier()
RF_model.fit(train_x, train_y)
RF_predictions = RF_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(RF_predictions, valid_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
RF_score = validation_score(RF_model, valid_x, valid_y)
print('Random Forest Cross Validation Score is: {0}'.format(RF_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.7104693140794224
Precision is: 0.7104693140794224
Recall is: 0.7104693140794224
F1 is: 0.7104693140794224
_____________
Random Forest Cross Validation Score is: 0.6245490564070483


In [ ]:
RF_predict_test = RF_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(RF_predict_test, test_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
RFtest_score = validation_score(RF_model, xtest, test_y)
print('Random Forest Cross Validation Score is: {0}'.format(RFtest_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.7403951701427003
Precision is: 0.7403951701427003
Recall is: 0.7403951701427003
F1 is: 0.7403951701427003
_____________
Random Forest Cross Validation Score is: 0.6553323725454873


# XGBoost

In [ ]:
from xgboost import XGBClassifier
XG_model = XGBClassifier()
XG_model.fit(train_x, train_y)
XG_predictions= XG_model.predict(valid_x)
accuracy, precision, recall, f1 = Metrics(XG_predictions, valid_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
XG_score = validation_score(XGBClassifier(), valid_x, valid_y)
print('XG Boost Cross Validation Score is {0}'.format(XG_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.636101083032491
Precision is: 0.636101083032491
Recall is: 0.636101083032491
F1 is: 0.636101083032491
_____________
XG Boost Cross Validation Score is 0.6180377437180691


In [ ]:
XG_predict_test= XG_model.predict(xtest)
accuracy, precision, recall, f1 = Metrics(XG_predict_test, test_y)
print("Accuracy is: {0}\nPrecision is: {1}\nRecall is: {2}\nF1 is: {3}".format(accuracy, precision, recall, f1))
print("_____________")
XGtest_score = validation_score(XGBClassifier(), xtest, test_y)
print('XG Boost Cross Validation Score is: {0}'.format(XGtest_score))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is: 0.6267837541163557
Precision is: 0.6267837541163557
Recall is: 0.6267837541163557
F1 is: 0.6267837541163557
_____________
XG Boost Cross Validation Score is: 0.6251035849396506


In [ ]:
The accuracy of the models as following

Navie Bayes - 77%

SVM - 78%

KNN - 72%

Decision Tree - 64%

Random Forest - 71%

XG Boost - 63.6%